In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import demoMF 
#import dask.dataframe as dd
#from sklearn.utils.extmath import randomized_svd
#from sklearn.decomposition import TruncatedSVD

In [2]:
# = "ml-25m/genome-scores.csv"
#gtpat
#h = "ml-25m/genome-tags.csv"
#path_tags = 'ml-/tags.csv'
path_ratings = 'ml-latest-small/ratings.csv'
path_movies = 'ml-latest-small/movies.csv'

In [3]:
#df_gscore = dd.read_csv(gspath)
#df_gtag = dd.read_csv(gtpath)
#df_tags = pd.read_csv(path_tags)
rating_df = pd.read_csv(path_ratings)
movie_df = pd.read_csv(path_movies)

In [4]:
combine_movie_rating = pd.merge(rating_df, movie_df, on='movieId')

In [5]:
columns = ['timestamp', 'genres']
combine_movie_rating = combine_movie_rating.drop(columns, axis=1)
combine_movie_rating.head(10)

,userId,movieId,rating,title
0,1,31,2.5,Dangerous Minds (1995)
1,7,31,3.0,Dangerous Minds (1995)
2,31,31,4.0,Dangerous Minds (1995)
3,32,31,4.0,Dangerous Minds (1995)
4,36,31,3.0,Dangerous Minds (1995)
5,39,31,3.0,Dangerous Minds (1995)
6,73,31,3.5,Dangerous Minds (1995)
7,88,31,3.0,Dangerous Minds (1995)
8,96,31,2.5,Dangerous Minds (1995)
9,110,31,4.0,Dangerous Minds (1995)


In [6]:
movie_ratingCount = (combine_movie_rating.
     groupby(by = ['movieId'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['movieId', 'totalRatingCount']]
    )

In [7]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'movieId', right_on = 'movieId', how = 'left')

In [8]:
user_rating = rating_with_totalRatingCount.drop_duplicates(['userId','movieId'])

In [9]:
movie_user_rating_pivot = user_rating.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)

In [10]:
movie_user_rating_pivot.to_csv("D:\Kaggle\VS\Git\Recommender-System\RecommenderPivot.csv")
movie_user_rating_pivot.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X = movie_user_rating_pivot.values
dim = movie_user_rating_pivot.shape
dim

(671, 9066)

In [12]:
objMF = demoMF.ExplicitMF(X,60,0.1,0.1,verbose=True)

In [13]:
objMF.train(2)

In [14]:
U = objMF.user_vecs
V = objMF.item_vecs
V.shape

(9066, 60)

In [15]:
matrix = V
corr = np.corrcoef(matrix)
#corr = pd.DataFrame(corr)
#corr.to_csv("D:\Kaggle\VS\Git\Recommender-System\Recommendercorr.csv", index= False)

In [16]:
#movie_title = movie_user_rating_pivot.columns
#movie_title_list = list(movie_title)
movie_ids = movie_user_rating_pivot.columns
movie_ids_list = list(movie_ids)

In [17]:
coffey_hands = movie_ids_list.index(1)

In [18]:
corr_coffey_hands  = list(corr[coffey_hands])
movie_ids = list(movie_ids)
#movie_title = list(movie_ids)
#corr_coffey_hands[:10]

In [19]:
from stopwatch import Stopwatch
watch = Stopwatch()

In [20]:
import random
 
def kthSmallest(arr, l, r, k):
    
    if (k > 0 and k <= r - l + 1):
        pos = randomPartition(arr, l, r)
 
        if (pos - l == k - 1):
            return arr[pos]
        if (pos - l > k - 1): 
            return kthSmallest(arr, l, pos - 1, k)
 
        return kthSmallest(arr, pos + 1, r,
                        k - pos + l - 1)
 
    return 999999999999
 
def swap(arr, a, b):
    temp = arr[a]
    arr[a] = arr[b]
    arr[b] = temp
 
def partition(arr, l, r):
    x = arr[r]
    i = l
    for j in range(l, r):
        if (arr[j] >= x):
            swap(arr, i, j)
            swap(movie_ids, i ,j)
            i += 1
    swap(arr, i, r)
    swap(movie_ids, i ,r)
    return i
def randomPartition(arr, l, r):
    n = r - l + 1
    pivot = int(random.random() * n)
    swap(arr, l + pivot, r)
    swap(movie_ids, l + pivot, r)
    return partition(arr, l, r)

In [21]:
k = 10
n_corr = len(corr_coffey_hands)-1
watch.reset()
watch.start()
selected = kthSmallest(corr_coffey_hands, 0, n_corr - 1, k)
watch.stop()
watch.duration

0.0018082999999933236

In [22]:
def quicksort(arr, start , stop):
    if(start < stop):
        pivotindex = partitionrand(arr, start, stop)
        quicksort(arr , start , pivotindex-1)
        quicksort(arr, pivotindex + 1, stop)
        
def partitionrand(arr , start, stop):
    randpivot = random.randrange(start, stop)
    arr[start], arr[randpivot] = arr[randpivot], arr[start]
    selected_movie_ids[start], selected_movie_ids[randpivot] = selected_movie_ids[randpivot], selected_movie_ids[start]
    return partition(arr, start, stop)

def partition(arr,start,stop):
    pivot = start
    i = start + 1
    for j in range(start + 1, stop + 1):
        if arr[j] <= arr[pivot]:
            arr[i] , arr[j] = arr[j] , arr[i]
            selected_movie_ids[i] , selected_movie_ids[j] = selected_movie_ids[j] , selected_movie_ids[i]
            i = i + 1
    arr[pivot] , arr[i - 1] = arr[i - 1] , arr[pivot]
    selected_movie_ids[pivot] , selected_movie_ids[i - 1] = selected_movie_ids[i - 1] , selected_movie_ids[pivot]
    pivot = i - 1
    return (pivot)

In [23]:
selected_corr = corr_coffey_hands[:k]
selected_movie_ids = movie_ids[:k]

In [24]:
watch.reset()
watch.start()
quicksort(selected_corr, 0, len(selected_corr) - 1)
watch.stop()
watch.duration

7.189999999468455e-05

In [25]:
selected_movie_ids

[4306, 364, 8961, 356, 1073, 260, 2355, 780, 3114, 1]

In [ ]:
'''
watch.reset()
watch.start()
k = 100
n_corr = len(corr_coffey_hands)-1
for i in range(n_corr, n_corr-k,-1):
    mx = 0
    for j in range(i+1):
        if (corr_coffey_hands[j]>=corr_coffey_hands[mx]):
            mx = j
    corr_coffey_hands[mx], corr_coffey_hands[i] = corr_coffey_hands[i], corr_coffey_hands[mx]
    #movie_ids[mx], movie_ids[i] = movie_ids[i], movie_ids[mx]
watch.stop()
watch.duration
'''

In [ ]:
#movie_ids[n_corr-10:n_corr]